In [1]:
from IPython.display import clear_output 
import cv2
import numpy as np
import mediapipe as mp
import time
import math

In [19]:
def pythagorean_theorem_dot_dist(coordinate_list):
    dot_distance = math.sqrt((coordinate_list[0].x - coordinate_list[1].x)**2 + (coordinate_list[0].y - coordinate_list[1].y)**2 + (coordinate_list[0].z - coordinate_list[1].z)**2)
    return dot_distance

def face_detection() :
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_face_detection = mp.solutions.face_detection
    mp_face_mesh = mp.solutions.face_mesh
    
    cap = cv2.VideoCapture(0)
    eye_closed_time = None
    user_score = 100
    
    while cap.isOpened() :
        success, image = cap.read()
        if not success:
            print("video is end.")
            break
            
        face_image = None
        
        with mp_face_detection.FaceDetection(
            model_selection=1, 
            min_detection_confidence=0.5) as face_detection :
            
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = face_detection.process(image)

            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            if results.detections :
                for detection in results.detections:
                    if detection.location_data:
                        box = detection.location_data.relative_bounding_box
                        ih, iw, _ = image.shape
                        x, y, w, h = int(box.xmin * iw), int(box.ymin * ih), int(box.width * iw), int(box.height * ih)
                        # Crop the face with margins
                        face_image = image[y:y+h, x:x+w]
                        face_image = cv2.resize(face_image, dsize=(500, 500), interpolation=cv2.INTER_AREA)
                        
                        with mp_face_mesh.FaceMesh(
                            max_num_faces=1,
                            refine_landmarks=True) as face_mesh :

                            face_image.flags.writeable = False
                            face_image = cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB)
                            results = face_mesh.process(face_image)
    
                            face_image.flags.writeable = True
                            face_image = cv2.cvtColor(face_image, cv2.COLOR_RGB2BGR)
                            if results.multi_face_landmarks :
                                LEFT_TOP_EYELID = 159
                                LEFT_BOTTOM_EYELID = 145
                            
                                RIGHT_TOP_EYELID = 386
                                RIGHT_BOTTOM_EYELID = 374
                                
                                for face_landmarks in results.multi_face_landmarks:
                                    mp_drawing.draw_landmarks(
                                        image=face_image,
                                        landmark_list=face_landmarks,
                                        connections=mp_face_mesh.FACEMESH_TESSELATION,
                                        landmark_drawing_spec=None,
                                        connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())
                                
                                    # 눈꺼풀의 위치를 시각화하기 위한 코드
                                    # Draw blue dots on the eyelids
                                    blue_dot = mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2)
                                    eyelids = [LEFT_TOP_EYELID, LEFT_BOTTOM_EYELID]
                                    for idx in eyelids:
                                        landmark_point = face_landmarks.landmark[idx]
                                        normalized_point = mp_drawing._normalized_to_pixel_coordinates(landmark_point.x, landmark_point.y, face_image.shape[1], face_image.shape[0])
                                        if normalized_point:
                                            cv2.circle(face_image, normalized_point, blue_dot.circle_radius, blue_dot.color, blue_dot.thickness)
                                    # 눈꺼풀의 위치를 시각화하기 위한 코드
                                    # Draw blue dots on the eyelids
                                    blue_dot = mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2)
                                    eyelids = [RIGHT_TOP_EYELID, RIGHT_BOTTOM_EYELID]
                                    for idx in eyelids:
                                        landmark_point = face_landmarks.landmark[idx]
                                        normalized_point = mp_drawing._normalized_to_pixel_coordinates(landmark_point.x, landmark_point.y, face_image.shape[1], face_image.shape[0])
                                        if normalized_point:
                                            cv2.circle(face_image, normalized_point, blue_dot.circle_radius, blue_dot.color, blue_dot.thickness)
                                    
                                    left_top_eyelid_coordinate = face_landmarks.landmark[LEFT_TOP_EYELID]
                                    left_bottom_eyelid_coordinate = face_landmarks.landmark[LEFT_BOTTOM_EYELID]
                                    
                                    right_top_eyelid_coordinate = face_landmarks.landmark[RIGHT_TOP_EYELID]
                                    right_bottom_eyelid_coordinate = face_landmarks.landmark[RIGHT_BOTTOM_EYELID]
                                    
                                    left_distance = pythagorean_theorem_dot_dist([left_top_eyelid_coordinate, left_bottom_eyelid_coordinate])
                                    right_distance = pythagorean_theorem_dot_dist([right_top_eyelid_coordinate, right_bottom_eyelid_coordinate])
                                    
                                    if left_distance <= 0.04 and right_distance <= 0.045 :
                                        if eye_closed_time is None :
                                            print(eye_closed_time)
                                            eye_closed_time = time.time()
                                        elif time.time() - eye_closed_time > 2 :
                                            print(time.time() - eye_closed_time)
                                            if user_score > 0 :
                                                user_score -= 2
                                            print(time.time() - eye_closed_time)
                                            eye_closed_time = None
                                    else :
                                        print("your eyes are opened")
                                    
                                    clear_output(wait=True)
                                    print(user_score)
                                    print(left_top_eyelid_coordinate)
                                    print(left_bottom_eyelid_coordinate)
                                    print(left_distance)
                                    print(right_distance)
                    else:
                        # Handle the case where no location data was found
                        # For example, print an error message and return None
                        print("No location data found for detection")
        if face_image is not None and face_image.shape[0] > 0 and face_image.shape[1] > 0:
            cv2.imshow('MediaPipe Hands', cv2.flip(face_image, 1))
        else:
            print("Face image is not valid.")

        if cv2.waitKey(5) & 0xFF == 27:
            break
            
    cap.release()
    cv2.destroyAllWindows()

In [20]:
face_detection()

100
x: 0.3168049454689026
y: 0.33747991919517517
z: -0.10037354379892349

x: 0.35411375761032104
y: 0.37307676672935486
z: -0.09268995374441147

0.05213559795284872
0.04717541604893748


In [10]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

# For webcam input:
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
cap = cv2.VideoCapture(0)
with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(image)

    # Draw the face mesh annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_face_landmarks:
      for face_landmarks in results.multi_face_landmarks:
        mp_drawing.draw_landmarks(
            image=image,
            landmark_list=face_landmarks,
            connections=mp_face_mesh.FACEMESH_TESSELATION,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_tesselation_style())
        mp_drawing.draw_landmarks(
            image=image,
            landmark_list=face_landmarks,
            connections=mp_face_mesh.FACEMESH_CONTOURS,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_contours_style())
        mp_drawing.draw_landmarks(
            image=image,
            landmark_list=face_landmarks,
            connections=mp_face_mesh.FACEMESH_IRISES,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_iris_connections_style())
    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Face Mesh', cv2.flip(image, 1))
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()